In [1]:
# %load_ext autoreload
# %autoreload 2

import os
import sys
os.environ['UTILS_PATH'] = r'C:\Users\Mi\Desktop\code\git\python-utils-ak'
os.environ['PROJECT_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat' 
os.environ['SQLITE_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\data.sqlite' 
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *

In [20]:
boiling_plan_df = pd.read_csv('2020.12.13 boiling_plan.csv')
boiling_plan_df['packing_left'] = boiling_plan_df['kg']
boiling_plan_df['sku'] = boiling_plan_df['sku'].apply(cast_sku)
# boiling_plan_df['boiling'] = boiling_plan_df['boiling'].apply(cast_boiling)
boiling_plan_df['boiling'] = boiling_plan_df['sku'].apply(lambda sku: sku.boilings[0])
boiling_plan_df.sort_values(by='id')
boiling_plan_df['speed'] = boiling_plan_df['sku'].apply(lambda sku: sku.packing_speed)
boiling_plan_df['bff'] = boiling_plan_df['sku'].apply(lambda sku:sku.boiling_form_factors[0])

boiling_plan_df.at[:2, 'packing_team_id'] = 1
boiling_plan_df.at[2:, 'packing_team_id'] = 2
boiling_plan_df.head()

,id,boiling,sku,kg,packed,speed,bff,packing_team_id
0,1.0,<Boiling 9>,<SKU 25>,1000.0,1000.0,960,<BoilingFormFactor 13>,1.0
1,2.0,<Boiling 9>,<SKU 25>,1000.0,1000.0,960,<BoilingFormFactor 13>,1.0
2,3.0,<Boiling 6>,<SKU 23>,192.8,192.8,960,<BoilingFormFactor 1>,2.0
3,3.0,<Boiling 6>,<SKU 26>,134.7,134.7,960,<BoilingFormFactor 1>,2.0
4,3.0,<Boiling 6>,<SKU 46>,153.6,153.6,960,<BoilingFormFactor 1>,2.0


In [3]:
boiling_grp = boiling_plan_df[boiling_plan_df['id'] == 3]

# mark groups
cur_group = 0
cur_form_factor = None
group_values = []
for i, row in boiling_grp.iterrows():
    if row['bff'] != cur_form_factor:
        cur_group += 1
        cur_form_factor = row['bff']
    group_values.append(cur_group)
boiling_grp['group'] = group_values

boiling_model = boiling_grp.iloc[0]['boiling']
boiling_grp

<ipython-input-3-174f4b55a90a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boiling_grp['group'] = group_values


,id,boiling,sku,kg,speed,bff,packing_team_id,group
2,3.0,<Boiling 6>,<SKU 23>,192.8,960,<BoilingFormFactor 1>,2.0,1
3,3.0,<Boiling 6>,<SKU 26>,134.7,960,<BoilingFormFactor 1>,2.0,1
4,3.0,<Boiling 6>,<SKU 46>,153.6,960,<BoilingFormFactor 1>,2.0,1
5,3.0,<Boiling 6>,<SKU 29>,155.3,702,<BoilingFormFactor 4>,2.0,2
6,3.0,<Boiling 6>,<SKU 31>,210.0,702,<BoilingFormFactor 4>,2.0,2
7,3.0,<Boiling 6>,<SKU 45>,153.6,1020,<BoilingFormFactor 4>,2.0,2


In [4]:
    # [packing.reconfiguration_times]
    def get_configuration_times(skus):
        res = []
        for i in range(len(skus) - 1):
            old_sku, sku = skus[i: i + 2]
            if old_sku is None:
                # todo: display first configuration?
                res.append(0)
            elif boiling_model.boiling_type == 'salt' and old_sku.weight_form_factor != sku.weight_form_factor and cast_packer(old_sku) == cast_packer(sku):
                # todo: take from parameters
                res.append(25)
            elif old_sku == sku:
                res.append(0)
            else:
                # todo: take from parameters
                res.append(5)
        return res

    
form_factor_label = gen_label([(sku.form_factor.name, sku.weight_form_factor) for sku, sku_kg in boiling_grp[['sku', 'kg']].values.tolist()])
brand_label = gen_label([(sku.brand_name, sku.weight_form_factor) for sku, sku_kg in boiling_grp[['sku', 'kg']].values.tolist()])

with make('melting', time_size=full_melting_time):
    if boiling_model.boiling_type == 'water':
        with make(h=1, push_func=dummy_push_y):
            make('serving', time_size=boiling_model.meltings.serving_time)
    with make(h=1, push_func=dummy_push_y):
        make('serving', time_size=boiling_model.meltings.serving_time, visible=False)
        make('melting_label', time_size=4 * 5)
        make('melting_name', time_size=full_melting_time - 4 * 5 - boiling_model.meltings.serving_time, form_factor_label=form_factor_label)
    with make(h=1, push_func=dummy_push_y):
        serving_visible = boiling_model.boiling_type == 'salt'
        make('serving', time_size=boiling_model.meltings.serving_time, visible=serving_visible)
        make('melting_process', time_size=melting_time, speed=boiling_model.meltings.speed)

        if boiling_model.boiling_type == 'salt':
            make('salting', time_size=boiling_model.meltings.salting_time)
    with make(h=1, push_func=dummy_push_y):
        make(time_size=boiling_model.meltings.serving_time, visible=False)
        if boiling_model.boiling_type == 'water':
            make('cooling1', time_size=boiling_model.meltings.first_cooling_time)
            make('cooling2', time_size=boiling_model.meltings.second_cooling_time)
        elif boiling_model.boiling_type == 'salt':
            make('salting', time_size=boiling_model.meltings.salting_time)

# prepare time
if boiling_model.boiling_type == 'water':
    prepare_time = boiling_model.meltings.serving_time + boiling_model.meltings.first_cooling_time + boiling_model.meltings.second_cooling_time
elif boiling_model.boiling_type == 'salt':
    prepare_time = boiling_model.meltings.serving_time + boiling_model.meltings.salting_time


IndentationError: expected an indented block (<ipython-input-4-0a9354b61779>, line 24)

In [5]:
# generate melting process
melting_events = []

maker = BlockMaker(default_push_func=dummy_push)
make = maker.make

with make('melting'):
    with make('servings', h=1, push_func=dummy_push_y):
        make('serving', time_size=boiling_model.meltings.serving_time)
    
    labels = make('labels', h=1, push_func=dummy_push_y).block
    meltings = make('meltings', h=1, push_func=dummy_push_y).block
    coolings = make('coolings', h=1, push_func=dummy_push_y).block
        
    for group, grp in boiling_grp.groupby('group'):
        if group > 1:
            # non-first group - reconfigure time
            dummy_push(meltings, Block('melting_configuration', size=1))
        
        melting_process = dummy_push(meltings, Block('melting_process', time_size=custom_round(grp['kg'].sum() / boiling_model.meltings.speed * 60, 5, 'ceil')))
        
        melting_events.append({'beg': melting_process.beg(), 
                               'time_size': grp['kg'].sum() / boiling_model.meltings.speed * 60, 
                               'speed': boiling_model.meltings.speed, 
                               'kg': grp['kg'].sum(),
                               'bff': grp.iloc[0]['bff']})

        # add cooling
        cooling_process = Block('cooling_process', t=melting_process.beg())
        cooling_maker = BlockMaker(cooling_process, default_push_func=dummy_push)
        cooling_make = cooling_maker.make
        
        with cooling_make('start'):
            if boiling_model.boiling_type == 'water':
                cooling_make('cooling', time_size=boiling_model.meltings.first_cooling_time)
                cooling_make('cooling', time_size=boiling_model.meltings.second_cooling_time)
            elif boiling_model.boiling_type == 'salt':
                cooling_make('cooling', time_size=boiling_model.meltings.salting_time)
        
        with cooling_make('finish'):
            cooling_make('cooling', time_size=melting_process.props['time_size'])
        
        add_push(coolings, cooling_process)

In [284]:
maker.root

root (0, 4] x (0, 30]
  melting (0, 4] x (0, 30]
    servings (0, 1] x (0, 6]
      serving (0, 1] x (0, 6]
    labels (1, 2] x (0, 0]
    meltings (2, 3] x (0, 15]
      melting_process (2, 3] x (0, 7]
      melting_configuration (2, 3] x (7, 8]
      melting_process (2, 3] x (8, 15]
    coolings (3, 4] x (0, 30]
      cooling_process (3, 4] x (0, 22]
        start (3, 4] x (0, 15]
          cooling (3, 4] x (0, 5]
          cooling (3, 4] x (5, 15]
        finish (3, 4] x (15, 22]
          cooling (3, 4] x (15, 22]
      cooling_process (3, 4] x (8, 30]
        start (3, 4] x (8, 23]
          cooling (3, 4] x (8, 13]
          cooling (3, 4] x (13, 23]
        finish (3, 4] x (23, 30]
          cooling (3, 4] x (23, 30]

In [272]:
melting_events

[{'beg': 0,
  'time_size': 32.073333333333345,
  'speed': 900,
  'kg': 481.10000000000025,
  'bff': <BoilingFormFactor 1>},
 {'beg': 8,
  'time_size': 34.59333333333332,
  'speed': 900,
  'kg': 518.8999999999997,
  'bff': <BoilingFormFactor 4>}]

In [17]:
def pack(melting_events, boiling_plan_df):
    # generate packings with event manager
    em = SimpleEventManager()

    # collect incoming speed of cheese
    class Meltings:
        def __init__(self):
            self.df = pd.DataFrame(columns=['speed', 'kg'])
            self.last_ts = None

        def on_melting(self, topic, ts, event):
            if event['bff'] not in self.df.index:
                self.df.at[event['bff'], 'kg'] = 0
            self.df.at[event['bff'], 'speed'] = event['speed']

        def update(self, topic, ts, event):
            if self.last_ts is None:
                self.last_ts = ts
                return

            self.df['kg'] += (ts - self.last_ts) / 3600 * self.df['speed']
            self.last_ts = ts


    meltings = Meltings()

    em.subscribe('', meltings.update)
    em.subscribe('melting', meltings.on_melting)

    for me in melting_events:
        em.add_event('melting', me['beg'] * 300, me)
        me = dict(me)
        me['speed'] = 0
        em.add_event('melting', me['beg'] * 300 + me['time_size'] * 60, me)

    for i in range(288):
        em.add_event('checkpoint', i * 300, {})


    class PackerTeam:
        def __init__(self, meltings, df, team_id):
            self.meltings = meltings
            self.df = df
            self.df['collected'] = 0.

            self.last_ts = None

            self.packing_start_ts = None
            self.packings = []

            self.team_id = team_id


        def update(self, topic, ts, event):
            df = self.df[self.df['packed'].abs() > 1e-2]
            df = df[df['packing_team_id'] == self.team_id]

            if len(df) == 0:
                return

            if not self.last_ts:
                self.last_ts = ts
                return

            cur_row = df.iloc[0] # pack one sku at one time
            if cur_row['bff'] in self.meltings.df.index:
                # take as much as we can
                available = self.meltings.df.at[cur_row['bff'], 'packed']
                needed = cur_row['packed']
                collected = min([(ts - self.last_ts) / 3600 * cur_row['speed'], 
                                 available,
                                 needed])
                self.df.at[cur_row.name, 'packed'] -= collected
                self.meltings.df.at[cur_row['bff'], 'kg'] -= collected
                self.df.at[cur_row.name, 'collected'] += collected

                if not self.packing_start_ts and collected:
                    # start packing
                    self.packing_start_ts = self.last_ts

            if abs(self.df.at[cur_row.name, 'packed']) < 1e-2:
                # finish packing 

                beg = self.packing_start_ts / 300
                end = ts / 300
                self.packings.append([beg, end, self.df.iloc[0]])
                self.packing_start_ts = None

            self.last_ts = ts

            # todo: add packing configuration

    packing_teams = {}
    for team_id in boiling_plan_df['packing_team_id'].unique():
        packing_teams[team_id] = PackerTeam(meltings, boiling_plan_df, team_id=team_id)
        em.subscribe('checkpoint', packing_team.update)

    em.run()
    return packing_teams

In [19]:
pack(melting_events, boiling_plan_df)

{1.0: <__main__.pack.<locals>.PackerTeam at 0x20feed71b80>,
 2.0: <__main__.pack.<locals>.PackerTeam at 0x20ff5e2e910>}

In [14]:
boiling_plan_df

,id,boiling,sku,kg,speed,bff,packing_team_id,collected
0,1.0,<Boiling 9>,<SKU 25>,1.000000e+03,960,<BoilingFormFactor 13>,1.0,0.0
1,2.0,<Boiling 9>,<SKU 25>,1.000000e+03,960,<BoilingFormFactor 13>,1.0,0.0
2,3.0,<Boiling 6>,<SKU 23>,0.000000e+00,960,<BoilingFormFactor 1>,2.0,192.8
3,3.0,<Boiling 6>,<SKU 26>,0.000000e+00,960,<BoilingFormFactor 1>,2.0,134.7
4,3.0,<Boiling 6>,<SKU 46>,8.526513e-14,960,<BoilingFormFactor 1>,2.0,153.6
5,3.0,<Boiling 6>,<SKU 29>,0.000000e+00,702,<BoilingFormFactor 4>,2.0,155.3
6,3.0,<Boiling 6>,<SKU 31>,0.000000e+00,702,<BoilingFormFactor 4>,2.0,210.0
7,3.0,<Boiling 6>,<SKU 45>,5.684342e-14,1020,<BoilingFormFactor 4>,2.0,153.6
8,4.0,<Boiling 9>,<SKU 30>,1.000000e+03,876,<BoilingFormFactor 4>,2.0,0.0
9,5.0,<Boiling 9>,<SKU 30>,1.000000e+03,876,<BoilingFormFactor 4>,2.0,0.0


In [277]:
maker = BlockMaker(default_push_func=dummy_push)
make = maker.make

with make('packing'):
    for beg, end, packing in packer_team1.packings:
        make('packing_process', t=beg, size=end - beg, push_fund=add_push)

In [278]:
maker.root

root (0, 1] x (0, 16]
  packing (0, 1] x (0, 16]
    packing_process (0, 1] x (0, 3]
    packing_process (0, 1] x (3, 5]
    packing_process (0, 1] x (5, 7]
    packing_process (0, 1] x (7, 10]
    packing_process (0, 1] x (10, 14]
    packing_process (0, 1] x (14, 16]

In [279]:
boiling_plan_df

,id,boiling,sku,kg,speed,bff,packing_team_id,collected
0,1.0,<Boiling 9>,<SKU 25>,1.000000e+03,960,<BoilingFormFactor 13>,1.0,0.0
1,2.0,<Boiling 9>,<SKU 25>,1.000000e+03,960,<BoilingFormFactor 13>,1.0,0.0
2,3.0,<Boiling 6>,<SKU 23>,0.000000e+00,960,<BoilingFormFactor 1>,2.0,192.8
3,3.0,<Boiling 6>,<SKU 26>,0.000000e+00,960,<BoilingFormFactor 1>,2.0,134.7
4,3.0,<Boiling 6>,<SKU 46>,8.526513e-14,960,<BoilingFormFactor 1>,2.0,153.6
5,3.0,<Boiling 6>,<SKU 29>,0.000000e+00,702,<BoilingFormFactor 4>,2.0,155.3
6,3.0,<Boiling 6>,<SKU 31>,0.000000e+00,702,<BoilingFormFactor 4>,2.0,210.0
7,3.0,<Boiling 6>,<SKU 45>,5.684342e-14,1020,<BoilingFormFactor 4>,2.0,153.6
8,4.0,<Boiling 9>,<SKU 30>,1.000000e+03,876,<BoilingFormFactor 4>,2.0,0.0
9,5.0,<Boiling 9>,<SKU 30>,1.000000e+03,876,<BoilingFormFactor 4>,2.0,0.0
